In [1]:
pip install --upgrade pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.8 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO

In [4]:
def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [5]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk import word_tokenize, pos_tag, ne_chunk

In [6]:
def getPhone(text):
    try:
        pattern = re.compile(r'([+(]?\d+[)\-]?[ \t\r\f\v]*[(]?\d{2,}[()\-]?[ \t\r\f\v]*\d{2,}[()\-]?[ \t\r\f\v]*\d*[ \t\r\f\v]*\d*[ \t\r\f\v]*)')
        phone = pattern.findall(text)
        phone = [re.sub(r'[,.]', '', el) for el in phone if len(re.sub(r'[()\-.,\s+]', '', el))>6]
        phone = [re.sub(r'\D$', '', el).strip() for el in phone]
        phone = [el for el in phone if len(re.sub(r'\D','',el)) <= 13 and len(re.sub(r'\D','',el))>=10]
        return phone[0]
    except:
        return None

In [7]:
def getEmail(text): 
    
    try:
        pattern = re.compile(r'\S*@\S*')
        email = pattern.findall(text)
        return email[0]
    except:
        return None

In [8]:
def getExperience(sentences):
    try:
        sen=[]
        z=0
        for words in sentences:
            for i in range(len(words)):
                if(words[i][0].lower()=='experience') or (words[i][0].lower()=='internships'):
                    index=[z,i]
                    break
            z+=1
        
        exp=[]
        for i in sentences[index[0]][index[1]+1:]:
            if i[0].isalpha() and i[1]=='NNP':
                exp.append(i[0])
        
        return exp        
        
    except:
        return None

In [9]:
def getQual(sentences):
    sen=[]
    z=0
    i2=[]
    i1=[]
    quals=['school','graduated','graduation','education','college','vidyalaya','vishwavidyalaya','university','mahavidyalaya','institute']
    for words in sentences:
        for i in range(len(words)):
            if(words[i][0].lower() in quals):
                i1.append(i)
                i2.append(z)
        z+=1

    i2=list(dict.fromkeys(i2))

    qual=[]
    for i in i2:
        if i==i2[0]:
            qual.append((" ".join([sentences[i][k][0] for k in range(i1[0]+1, len(sentences[i]))])))
        else:
            qual.append((" ".join([sentences[i][k][0] for k in range(len(sentences[i]))])))
        
        
    return qual

In [10]:
def getSkills(sentences):
    try:
        sen=[]
        z=0
        for words in sentences:
            for i in range(len(words)):
                if(words[i][0].lower()=='skills') and words[i][1]=='NNP':
                    index =[z,i]
                    break
            z+=1

        skills=[]
        for i in sentences[index[0]][index[1]+1:]:
            if i[0].isalpha() and i[1]=='NNP':
                skills.append(i[0])

        return skills
    except:
        return None

In [11]:
def getCertification(sentences):
    try:
        sen=[]
        z=0
        global index
        for words in sentences:
            for i in range(len(words)):
                if(words[i][0].lower()=='certifications'):
                    index=[z,i]
                    break
            z+=1

        certis=" ".join([sentences[index[0]][k][0] for k in range(1,len(sentences[index[0]]))])

        return certis
    except:
        return None

In [12]:
def getDetails(text):
    
    sentences=nltk.sent_tokenize(text)
    sentences=[nltk.word_tokenize(el) for el in sentences]
    sentences=[nltk.pos_tag(el) for el in sentences]
    phone=getPhone(text)
    mail=getEmail(text)
    exp=getExperience(sentences)
    quals=getQual(sentences)
    skills=getSkills(sentences)
    certis=getCertification(sentences)
    dict={
        'Phone_no': phone,
        'Email':mail,
        'Experience': exp,
        'Qualification':quals,
        'Skills': skills,
        'Certifications': certis
    }
    return dict

In [14]:
text=convert_pdf_to_txt("/kaggle/input/re-1234/Resume_Suresh_Pawar.pdf")   # Enter the path to the resume here

In [15]:
details=getDetails(text)

In [16]:
details.keys()

dict_keys(['Phone_no', 'Email', 'Experience', 'Qualification', 'Skills', 'Certifications'])

In [18]:
details

{'Phone_no': '2014679715',
 'Email': 'sureshkumar.pawar@gmail.com',
 'Experience': [],
 'Qualification': ['Bachelor of Computer Science and Engineering , SSGM College Of Engineering 0 7 / 2 0 0 0 - 0 7 / 2 0 0 4 LINKS LinkedIn'],
 'Skills': None,
 'Certifications': None}